<a href="https://colab.research.google.com/github/WellcomePeujio/Prediccion-de-Precio-del-Oro-con-Machine-Learning/blob/main/Predicci%C3%B3n_de_Precio_del_Oro_con_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de Precio del Oro con Machine Learning

**Título**: "¿Podemos predecir el precio del oro?"

**Problema**: Se tienen los precios del oro en los últimos **5 años**, junto con variables macroeconómicas como:

- **Inflación**
- **Tasa de interés**
- **Precio del dólar**

El objetivo es entrenar un modelo de **Machine Learning** para predecir el precio del oro y evaluar su desempeño.

**Preguntas**:

1. **Entrena un modelo de regresión lineal o XGBoost** para predecir el precio del oro.
2. **Evalúa la precisión del modelo** con el **Error Cuadrático Medio (RMSE)**.
3. **Compara los resultados** con un modelo basado en **ARIMA**.
4. **Visualización**: Genera un **gráfico comparando el precio real vs. el modelo de predicción**.


# Resolución Matemática: Predicción de Precio del Oro con Machine Learning

## Paso 1: Definición del Problema

Queremos predecir el precio del oro $(P_{\text{oro}})$ en función de variables económicas como:

- **Inflación** $(\text{INF})$
- **Tasa de interés** $(\text{TI})$
- **Precio del dólar** $(\text{USD})$

Utilizamos un modelo de **regresión lineal** para establecer la relación:

$$
P_{\text{oro}, t} = \beta_0 + \beta_1 \text{INF}_t + \beta_2 \text{TI}_t + \beta_3 \text{USD}_t + \epsilon_t
$$

Donde:
- $\beta_0$ es el intercepto.
- $\beta_1, \beta_2, \beta_3$ son los coeficientes de regresión.
- $\epsilon_t$ es el error del modelo.

También probamos un modelo de **XGBoost**, que captura relaciones no lineales en los datos.

---

## Paso 2: Evaluación del Modelo con RMSE

El **Error Cuadrático Medio (RMSE)** mide la precisión de las predicciones:

$$
RMSE = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (P_{\text{oro, real}, i} - P_{\text{oro, predicho}, i})^2}
$$

Un **RMSE bajo** indica un mejor ajuste del modelo.

---

## Paso 3: Comparación con ARIMA

El modelo **ARIMA (p, d, q)** captura patrones temporales en la serie del precio del oro. Su ecuación general es:

$$
Y_t = c + \phi_1 Y_{t-1} + \phi_2 Y_{t-2} + ... + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \epsilon_t
$$

Donde:
- $p$ = número de rezagos autoregresivos.
- $d$ = número de diferencias aplicadas para hacer la serie estacionaria.
- $q$ = número de rezagos en el término de error.

---

## Paso 4: Conclusiones

1. Entrenamos modelos de **regresión lineal y XGBoost** para predecir el precio del oro.
2. Evaluamos su desempeño con **RMSE**.
3. Comparamos con un **modelo ARIMA** para verificar si una estrategia basada solo en series temporales es más precisa.
4. Generamos una **visualización** comparando los valores reales y predichos.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# Simulación de datos para el precio del oro y variables económicas
np.random.seed(42)
dates = pd.date_range(start="2019-01-01", periods=1250, freq="B")  # 5 años de datos bursátiles
inflation = np.random.uniform(1.5, 3.5, len(dates))  # Simulación de inflación entre 1.5% y 3.5%
interest_rate = np.random.uniform(0.5, 2.5, len(dates))  # Simulación de tasas de interés entre 0.5% y 2.5%
usd_price = np.random.uniform(90, 110, len(dates))  # Simulación del precio del USD

# Precio del oro simulado basado en las variables económicas + ruido
gold_price = 1500 + 10 * inflation - 5 * interest_rate + 3 * usd_price + np.random.normal(0, 30, len(dates))

# Crear DataFrame
data = pd.DataFrame(index=dates, data={
    "Gold_Price": gold_price,
    "Inflation": inflation,
    "Interest_Rate": interest_rate,
    "USD_Price": usd_price
})

# División de datos en entrenamiento y prueba
X = data[["Inflation", "Interest_Rate", "USD_Price"]]
y = data["Gold_Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Modelo de Regresión Lineal
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin = lin_reg.predict(X_test)
rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))

# Modelo XGBoost
xgb_model = XGBRegressor(objective="reg:squarederror", n_estimators=100)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
# Import necessary libraries for ARIMA model
from statsmodels.tsa.arima.model import ARIMA

# Preparar datos para ARIMA (usamos solo la serie temporal de precios del oro)
train_size = int(len(data) * 0.8)
train, test = data["Gold_Price"][:train_size], data["Gold_Price"][train_size:]

# Ajustar modelo ARIMA (p=3, d=1, q=2 como valores iniciales)
arima_model = ARIMA(train, order=(3, 1, 2))
arima_fitted = arima_model.fit()

# Generar predicciones para el conjunto de prueba
arima_pred = arima_fitted.forecast(steps=len(test))

# Calcular RMSE para ARIMA
rmse_arima = np.sqrt(mean_squared_error(test, arima_pred))

# Gráfico de comparación con ARIMA
plt.figure(figsize=(12, 6))
plt.plot(test.index, test, label="Precio Real", color="black")
plt.plot(test.index, y_pred_lin, label=f"Regresión Lineal (RMSE={rmse_lin:.2f})", color="blue", linestyle="--")
plt.plot(test.index, y_pred_xgb, label=f"XGBoost (RMSE={rmse_xgb:.2f})", color="red", linestyle="--")
plt.plot(test.index, arima_pred, label=f"ARIMA (RMSE={rmse_arima:.2f})", color="green", linestyle="--")

plt.xlabel("Fecha")
plt.ylabel("Precio del Oro ($)")
plt.title("Predicción del Precio del Oro: Regresión Lineal vs. XGBoost vs. ARIMA")
plt.legend()
plt.grid(True)
plt.show()